<a href="https://colab.research.google.com/github/jemelike/spark_snippets/blob/main/aggregate_multiple_pandas_dataframes_into_a_single_spark_dataframe_via_unionbyname.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas numpy pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy   as np
import pandas  as pd
import pyspark as spark

In [ ]:
def generate_pandas_df(max_num_columns:int=10, max_num_rows:int=10)-> pd.DataFrame:
  num_cols = np.random.randint(1, high=max_num_columns)
  num_rows = np.random.randint(1, high=max_num_rows)

  return pd.DataFrame(np.random.randn(num_rows, num_cols)).copy()

In [ ]:
def pad_pandas_df(df: pd.DataFrame, total_field: int, prefix:str="FIELD_") -> pd.DataFrame:
  num_rows, num_col = df.shape
  new_df = df.copy()
  mapping = { k: f"{prefix}{idx}" for idx, k in enumerate(new_df.columns) }
  new_df = new_df.rename(columns=mapping)

  for i in np.arange(len(mapping), total_field):
    padded_col = f"{prefix}{i}"
    new_df[padded_col] = [None] * num_rows
  return new_df

In [ ]:
def build_generic_schema(df:pd.DataFrame)->pd.DataFrame:
  return spark.sql.types.StructType([ spark.sql.types.StructField(col,spark.sql.types.StringType(),True) for col in df.columns ])

In [ ]:
# Generate pd.DataFrames
pandas_dataframes = [generate_pandas_df() for i in range(250)]

# Determine the maximum number of columns
NUM_FIELDS      = max([ df.shape[1] for df in pandas_dataframes])
TOTAL_ROW_COUNT = sum([ df.shape[0] for df in pandas_dataframes])
TOTAL_COL_COUNT = NUM_FIELDS + 1 # Accounts for the added column
# Pad Datafranes
padded_pandas_dataframes = [pad_pandas_df(df, NUM_FIELDS) for df in pandas_dataframes]
# del pandas_dataframes


In [ ]:
assert len(set([ df.shape[1] for df in padded_pandas_dataframes])) == 1, "DataFrames do not contain the same number of columns."

In [ ]:
spark_session = spark.sql.SparkSession.builder.master("local").appName("app-unionByName-exercise").getOrCreate()


In [ ]:
# Test
spark_dataframes =[spark_session.createDataFrame(df, schema=build_generic_schema(df)).selectExpr(f"'table_no_{table_no}' as TABLE_no",'*') for table_no,df in enumerate(padded_pandas_dataframes)]
del padded_pandas_dataframes

In [ ]:
main_df = None
for union_count, df in enumerate(spark_dataframes):
  if main_df is None:
    main_df = df.alias("main")
  else:
    main_df = df.unionByName(main_df).alias(f"main_{union_count}")
  print(f"Union {union_count}")

In [ ]:
main_df.show()

+------------+--------------------+-------------------+--------------------+-------------------+-------------------+-------------------+------------------+--------------------+-------+
|    TABLE_no|             FIELD_0|            FIELD_1|             FIELD_2|            FIELD_3|            FIELD_4|            FIELD_5|           FIELD_6|             FIELD_7|FIELD_8|
+------------+--------------------+-------------------+--------------------+-------------------+-------------------+-------------------+------------------+--------------------+-------+
|table_no_249| -0.9505574125247941| 0.5391903961618045| -0.6815839660129692| 1.3091440803905519|               null|               null|              null|                null|   null|
|table_no_249|  1.4502969521656452|-0.2824001578351148|   2.084311639050579| 1.2200922181393752|               null|               null|              null|                null|   null|
|table_no_248| -1.1661180190870344| 1.1363926436977319| -0.8492702906038235

In [ ]:
assert main_df.count() == TOTAL_ROW_COUNT, "Missing rows from union"

In [ ]:
assert len(main_df.columns) == TOTAL_COL_COUNT, "Missing columns from union"